In [1]:
from datasets import load_dataset
import json

DATASET_NAME = "arc_ir"
orders = ["train", "validation", "test"]
dataset = {}
dataset["easy"] = load_dataset("allenai/ai2_arc", "ARC-Easy", split=orders)
dataset["challenge"] = load_dataset("allenai/ai2_arc",
                                    "ARC-Challenge",
                                    split=orders)

Generating train split:   0%|          | 0/2251 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2376 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/570 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1119 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1172 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/299 [00:00<?, ? examples/s]

In [2]:
import pandas as pd
PARENT='ARC-OBQA-RegLivEnv-IR10V8'
data_ir=[pd.read_json(f'{PARENT}/{x}.jsonl', lines=True) for x in ['train','dev','test']]

In [3]:
[x[1]['para'] for x in data_ir[0].iterrows()  if 'form neutron star' in x[1]['para'] ]

['It is commonly believed that the most massive stars form black holes, while massive stars with a mass below a certain limit ( ) form neutron stars. neutron star mass limit Black holes - mass limit for neutron stars; Black Holes Now it is reasonable that neutron stars have the same fate awaiting them as white dwarfs if their mass is increased arbitrarily. High mass stars will explode and their cores will form neutron stars or black holes. Depending on the initial mass of the star, a neutron star or a black hole is formed. The very massive stars (20 times the mass of the sun or more) probably turn in to black holes, while stars with 8 to 20 times the mass of the sun become neutron stars. but the other is a collapsed star, such as a neutron star or a black hole, which has about the same mass as our Sun but has shrunk to ten kilometers or less in radius. Temperature scale on H-R diagram Mass-Luminosity formula Mass of stars and life cycles Defining supernova Defining Quasars Quasar power

In [4]:
dataset["challenge"][2][0]

{'id': 'Mercury_7175875',
 'question': 'An astronomer observes that a planet rotates faster after a meteorite impact. Which is the most likely effect of this increase in rotation?',
 'choices': {'text': ['Planetary density will decrease.',
   'Planetary years will become longer.',
   'Planetary days will become shorter.',
   'Planetary gravity will become stronger.'],
  'label': ['A', 'B', 'C', 'D']},
 'answerKey': 'C'}

In [5]:
# from tqdm.notebook import tqdm

# parent = "ARC-DA-v1.1"
# data_ir = [[] for x in orders]
# for idx, order in enumerate(tqdm(orders)):
#     for a in tqdm(open(f"{parent}/{order}.jsonl")):
#         data_ir[idx].append(json.loads(a))

In [6]:
# ['easy' if 'ARCEZ' in ques_id else 'challenge']
clue_map = {}


def cleanit(x):
    if len(x)>0 and x[-1] == ".":
        return x[:-1]
    return x

from tqdm.notebook import tqdm, trange
for portion_idx in [0, 1, 2]:
    for clue in tqdm(data_ir[portion_idx].iterrows()):
        row=clue[1]
        xx=cleanit(row['para'])
        if '( )' in xx:
            # print(xx)
            xx=xx.replace('( )','')
        clue_map[row['id'][row['id'].index("_") + 1:]] = xx

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [7]:
NUM_PERMUTE_QUESTION = None
EACH_HAS = 50
NUM_CHOICE = 4

In [8]:
dataset["easy"][2]["choices"]

[{'text': ['Sunlight is the source of energy for nearly all ecosystems.',
   'Most ecosystems are found on land instead of in water.',
   'Carbon dioxide is more available than other gases.',
   'The producers in all ecosystems are plants.'],
  'label': ['A', 'B', 'C', 'D']},
 {'text': ['safety goggles', 'breathing mask', 'rubber gloves', 'lead apron'],
  'label': ['A', 'B', 'C', 'D']},
 {'text': ['brain cells', 'bone cells', 'muscle cells', 'ovary cells'],
  'label': ['A', 'B', 'C', 'D']},
 {'text': ['gray', 'warm', 'long', 'soft'], 'label': ['A', 'B', 'C', 'D']},
 {'text': ['a lightweight core surrounded by neutral particles',
   'a massive core surrounded by negatively-charged particles',
   'a network of interacting positive and negative particles',
   'overlapping layers of neutral, positive, and negative particles'],
  'label': ['A', 'B', 'C', 'D']},
 {'text': ['meter.', 'kilometer.', 'light-year.', 'astronomical unit.'],
  'label': ['A', 'B', 'C', 'D']},
 {'text': ['wash hands',

In [9]:
for _, data in dataset.items():
    arr = [len(x["label"]) for x in data[2]["choices"]]
    choice_len = max(arr)
    print("max", choice_len)
    print("min", min(arr))
    print("avg", sum(arr) / len(arr))

max 5
min 3
avg 3.9987373737373737
max 5
min 3
avg 3.999146757679181


In [10]:
import numpy as np

def get_clue(json_line):
    x = json_line["id"]
    if x in clue_map:
        return clue_map[x]
    return None
def get_prompt(dataset, ques_index, location=-1, has_choice=False):
    # try:
    json_line = dataset[ques_index]
    question = json_line["question"]
    choices = json_line["choices"]
    choice_texts = choices["text"].copy()
    answer_key = json_line["answerKey"][0]
    answer_key_idx = ord(answer_key) - (ord("A") if answer_key in "ABCDE" else ord("1"))
    answer_text = choice_texts[answer_key_idx]
    if location > -1:
        perm = np.random.permutation(len(choice_texts) - 1)
        choice_texts.remove(answer_text)
        choice_texts = [
            choice_texts[perm[i]].replace("\n", " ") for i in range(len(choice_texts))
        ]
        choice_texts.insert(location, answer_text)
    else:
        perm = list(range(len(choice_texts)))
        choice_texts = [
            choice_texts[perm[i]].replace("\n", " ") for i in range(len(choice_texts))
        ]
    candidates = " ".join(
        [
            f"( ) {cleanit(text)}"
            for text in choice_texts
        ]
    ).replace("\n", " ")
    clue = get_clue(json_line)
    
    prompt = f"Clue: {clue}. Question: {question} \\n {candidates}"
    # if clue:
    #     print(prompt,'Answer: ', answer_text)
    return prompt, cleanit(answer_text)

import pickle
if NUM_PERMUTE_QUESTION:
    each_choice_has = EACH_HAS / NUM_CHOICE
    for key, datas in dataset.items():
        container = []
        count = 0
        for ques_index in trange(len(datas[2])):
            if count == NUM_PERMUTE_QUESTION:
                break
            if len(datas[2][ques_index]["choices"]["text"]) == NUM_CHOICE:
                count += 1
                for idx in range(EACH_HAS):
                    location = int(idx / each_choice_has)
                    container.append(
                        get_prompt(datas[2], ques_index, location, has_choice=False)
                    )
        container1 = container
        pickle.dump(
            (count, EACH_HAS, container),
            open(f"{DATASET_NAME}_{key}_test_permute.pkl", "wb"),
        )
        print(count)
else:
    container_train=[]
    for key, datas in dataset.items():
        container_train.extend([get_prompt(datas[0], idx, has_choice=False) for idx in trange(len(datas[0]))])
        
        # container_train.extend(
        #     [get_prompt(datas[1], idx, has_choice=False) for idx in trange(len(datas[1]))]
        # )
    pickle.dump(container_train, open(f"{DATASET_NAME}_combine_train.pkl", "wb"))
    for key, datas in dataset.items():
        pickle.dump(
            [
                get_prompt(datas[2], idx, has_choice=False)
                for idx in trange(len(datas[2]))
            ],
            open(f"{DATASET_NAME}_{key}_test.pkl", "wb"),
        )

  0%|          | 0/2251 [00:00<?, ?it/s]

  0%|          | 0/1119 [00:00<?, ?it/s]

  0%|          | 0/2376 [00:00<?, ?it/s]

  0%|          | 0/1172 [00:00<?, ?it/s]

In [11]:
[x for x in container_train if 'form neutron star' in x[0]]

[('Clue: It is commonly believed that the most massive stars form black holes, while massive stars with a mass below a certain limit  form neutron stars. neutron star mass limit Black holes - mass limit for neutron stars; Black Holes Now it is reasonable that neutron stars have the same fate awaiting them as white dwarfs if their mass is increased arbitrarily. High mass stars will explode and their cores will form neutron stars or black holes. Depending on the initial mass of the star, a neutron star or a black hole is formed. The very massive stars (20 times the mass of the sun or more) probably turn in to black holes, while stars with 8 to 20 times the mass of the sun become neutron stars. but the other is a collapsed star, such as a neutron star or a black hole, which has about the same mass as our Sun but has shrunk to ten kilometers or less in radius. Temperature scale on H-R diagram Mass-Luminosity formula Mass of stars and life cycles Defining supernova Defining Quasars Quasar p